# Data Collection: 

## 3. Scraping Lyrics from Genius Lyrics API

This script removes "/" and "(" in song names, as well as the word "featuring" in artist names, and then iterate through all the song names and artist names and get the lyrics using Genius API.

# deleted, deleted1, deleted3:

This function removes "/", "(" and the word "featuring".

# get_lyrics, get_lyrics2:
get_lyrics gets the first 4826 lyrics, because the 4827th song's link is broken. get_lyrics2 scrapes from the 4828th song.

The lyrics were put into two lists and combined together.

# to_clean:
After adding lyrics to the top_100 dataframe as a column, this function cleans the lyrics.

In [2]:
import requests
import pandas as pd
import re
import numpy as np

#!pip install lyricsgenius
from lyricsgenius import Genius
cid ='1cb187b75d4645df8a7d239c3ffXXXXX'
secret = '917a6d53b558462f94ace155312XXXXX'
token = 'p2u3klJkdZZ1-eqnm_9WVuCY3vVcGyhZzwulIIEJzXCdZTB7MfBK8iuaOTpXXXXX'

In [4]:
top_100= pd.read_csv("../data/top100_1958_to_2021.csv")

In [3]:
lyrics = []

def get_lyrics(token, end_df):
    genius = Genius(token)
    genius.timeout = 15
    genius.sleep_time = 0.4
    for track in end_df.values:
        retries = 0
        while retries < 3:
            try:
                song = genius.search_song(track[2], track[3])
            except Timeout as e:
                retries += 1
                continue
            if song is not None:
                lyrics.append(song.lyrics)
            else:
                lyrics.append(np.NAN)
            break
            
def deleted(string):
    head, sep, tail = string.partition('/')
    return head
def deleted1(string):
    head, sep, tail = string.partition('(')
    if head[-1] == " ":
        result = head[:-1] + '"'
    else:
        result = head
    return result
def deleted3(string):
    head, sep, tail = string.partition('f')
    return head

In [4]:
for i in top_100.index:
    if "/" in top_100.iloc[i][2]:
        top_100.at[i, 'song'] = deleted(top_100.iloc[i][2])
    if "(" in top_100.iloc[i][2]:
        top_100.at[i, 'song'] = deleted1(top_100.iloc[i][2])
for i in top_100.index:
    if "featuring" in top_100.iloc[i][3]:
        top_100.at[i, 'artist(s)'] = deleted3(top_100.iloc[i][3])

In [5]:
top_100

,no.,year,song,artist(s),song_wiki_url,artist_wiki_url
0,1,1958,"""Volare""",Domenico Modugno,['https://en.wikipedia.org/wiki/Volare_(song)'],['https://en.wikipedia.org/wiki/Domenico_Modug...
1,2,1958,"""All I Have to Do Is Dream""",The Everly Brothers,['https://en.wikipedia.org/wiki/All_I_Have_to_...,['https://en.wikipedia.org/wiki/The_Everly_Bro...
2,3,1958,"""Don't""",Elvis Presley,['https://en.wikipedia.org/wiki/Don%27t_(Elvis...,['https://en.wikipedia.org/wiki/Elvis_Presley']
3,4,1958,"""Witch Doctor""",David Seville,['https://en.wikipedia.org/wiki/Witch_Doctor_(...,['https://en.wikipedia.org/wiki/Ross_Bagdasari...
4,5,1958,"""Patricia""",Perez Prado,['https://en.wikipedia.org/wiki/Patricia_(Pere...,['https://en.wikipedia.org/wiki/Perez_Prado']
...,...,...,...,...,...,...
6346,96,2021,"""Things a Man Oughta Know""",Lainey Wilson,['https://en.wikipedia.org/wiki/Things_a_Man_O...,['https://en.wikipedia.org/wiki/Lainey_Wilson']
6347,97,2021,"""Throat Baby""",BRS Kash,['https://en.wikipedia.org/wiki/Throat_Baby_(G...,['https://en.wikipedia.org/wiki/BRS_Kash']
6348,98,2021,"""Tombstone""",Rod Wave,['https://en.wikipedia.org/wiki/Tombstone_(son...,['https://en.wikipedia.org/wiki/Rod_Wave']
6349,99,2021,"""Drinkin' Beer. Talkin' God. Amen.""",Chase Rice,['https://en.wikipedia.org/wiki/Drinkin%27_Bee...,"['https://en.wikipedia.org/wiki/Chase_Rice', '..."


In [6]:
#don't run this, takes forever to run
get_lyrics(token,top_100)  #will got stuck at the 4827th row, because there's an error in this song in Genius

In [8]:
lyrics2 = []

def get_lyrics2(token, end_df):
    genius = Genius(token)
    genius.timeout = 15
    genius.sleep_time = 1  
    for track in end_df.values:
        retries = 0
        while retries < 3:
            try:
                song = genius.search_song(track[2], track[3])
            except Timeout as e:
                retries += 1
                continue
            if song is not None:
                lyrics2.append(song.lyrics)
            else:
                lyrics2.append(np.NAN)
            break

In [9]:
#don't run this either
get_lyrics2(token, top_100[4828:]) #continue scraping from the 4828th row

In [10]:
lyrics_final = lyrics + [np.NAN] + lyrics2

In [11]:
top_100['lyrics'] = lyrics_final

In [40]:
top_100

,Unnamed: 0,no.,year,song,artist(s),song_wiki_url,artist_wiki_url,lyrics
0,0,1,1958,"""Volare""",Domenico Modugno,['https://en.wikipedia.org/wiki/Volare_(song)'],['https://en.wikipedia.org/wiki/Domenico_Modug...,Volare LyricsPenso che un sogno cosi` non rito...
1,1,2,1958,"""All I Have to Do Is Dream""",The Everly Brothers,['https://en.wikipedia.org/wiki/All_I_Have_to_...,['https://en.wikipedia.org/wiki/The_Everly_Bro...,All I Have to Do Is Dream Lyrics[Intro]\nDream...
2,2,3,1958,"""Don't""",Elvis Presley,['https://en.wikipedia.org/wiki/Don%27t_(Elvis...,['https://en.wikipedia.org/wiki/Elvis_Presley'],"Don’t Lyrics[Verse 1]\nDon't, don't, that's wh..."
3,3,4,1958,"""Witch Doctor""",David Seville,['https://en.wikipedia.org/wiki/Witch_Doctor_(...,['https://en.wikipedia.org/wiki/Ross_Bagdasari...,Witch Doctor Lyrics{Movie version -\nDJ:\nYeah...
4,4,5,1958,"""Patricia""",Perez Prado,['https://en.wikipedia.org/wiki/Patricia_(Pere...,['https://en.wikipedia.org/wiki/Perez_Prado'],Patricia LyricsKiss her and your lips will alw...
...,...,...,...,...,...,...,...,...
6346,6346,96,2021,"""Things a Man Oughta Know""",Lainey Wilson,['https://en.wikipedia.org/wiki/Things_a_Man_O...,['https://en.wikipedia.org/wiki/Lainey_Wilson'],Things a Man Oughta Know Lyrics[Verse 1]\nI ca...
6347,6347,97,2021,"""Throat Baby""",BRS Kash,['https://en.wikipedia.org/wiki/Throat_Baby_(G...,['https://en.wikipedia.org/wiki/BRS_Kash'],Throat Baby (Go Baby) Lyrics[Intro]\n(What's h...
6348,6348,98,2021,"""Tombstone""",Rod Wave,['https://en.wikipedia.org/wiki/Tombstone_(son...,['https://en.wikipedia.org/wiki/Rod_Wave'],"Tombstone Lyrics[Intro]\nDamn, this motherfuck..."
6349,6349,99,2021,"""Drinkin' Beer. Talkin' God. Amen.""",Chase Rice,['https://en.wikipedia.org/wiki/Drinkin%27_Bee...,"['https://en.wikipedia.org/wiki/Chase_Rice', '...",Drinkin’ Beer. Talkin’ God. Amen. Lyrics[Verse...


In [12]:
top_100.to_csv('../data/top_100_with_lyrics.csv')

In [13]:
top_100 = pd.read_csv('../data/top_100_with_lyrics.csv')

In [14]:
top_100.dropna(subset = ["lyrics"], inplace=True)

In [7]:
def to_clean(item):
    no_n = item.replace('\n',' ')
    no_embed = no_n[:-5]

    no_p = no_embed.replace('.',' ')
    no_c = no_p.replace(',',' ')
    no_pa1 = no_c.replace('(',' ')

    no_pa2 = no_pa1.replace(')',' ')
    no_brace1 = no_pa2.replace('{',' ')
    no_brace2 = no_brace1.replace('}',' ')
    no_L = re.sub(r'.*Lyrics', '', no_brace2)
    no_s = re.sub("\[.*?\]","",no_L)
    no_num = re.sub("\d", "", no_s)
    return no_num

In [9]:
top_100['lyrics'] = top_100['lyrics'].apply(to_clean)

In [10]:
top_100.to_csv('../data/cleaned_lyrics.csv')